In [74]:

# Importing Libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
%matplotlib inline
import os
import time

# Import keys
from api_keys import g_key
from api_keys import weather_api_key


In [75]:
csv_file = os.path.join('..', 'Output', 'cities.csv')

citiesweather_df = pd.read_csv(csv_file)
citiesweather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


In [76]:
# Configuring gmaps with api_key
gmaps.configure(api_key=g_key)

# Creating latitudes and longitudes under "Lat" and "Lng"
locations = citiesweather_df[["Lat", "Lng"]]

# Creating humidities under 'Humidity'
humidity = citiesweather_df["Humidity"]

In [77]:
# Plotting the heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Creating the heatmap layers
heatmap = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Adding a layer
fig.add_layer(heatmap)

# Displaying heatmap
fig

Figure(layout=FigureLayout(height='420px'))

In [78]:
max_intensity

290

In [79]:
# Filtering out cities with a wind speed of less than 15 mph, no clouds and max temp between 70 and 80
filteredcities_df = citiesweather_df.loc[(citiesweather_df["Wind Speed"] <= 10) & (citiesweather_df["Cloudiness"] == 0) & \
                                   (citiesweather_df["Max Temp"] >= 70) & (citiesweather_df["Max Temp"] <= 80)].dropna()

filteredcities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [80]:
# Creating a hotel dataframe
hotels_df = filteredcities_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotels_df["Hotel Name"] = ""

# Display the result
hotels_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
297,mogok,MM,22.92,96.51,
444,birjand,IR,32.86,59.22,


In [81]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [82]:
for index, row in filteredcities_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotels_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 70: ponta do sol.
Closest hotel in ponta do sol is Hotel Escarpas do Lago.
------------
Retrieving Results for Index 88: cidreira.
Closest hotel in cidreira is Hotel Castelo.
------------
Retrieving Results for Index 138: morondava.
Closest hotel in morondava is Kimony Resort Hotel.
------------
Retrieving Results for Index 139: vaini.
Closest hotel in vaini is Dandeli Aura Jungle Stay.
------------
Retrieving Results for Index 176: nador.
Closest hotel in nador is Hotel Maghreb El Arabi.
------------
Retrieving Results for Index 297: mogok.
Closest hotel in mogok is Mount Mogok Hotel.
------------
Retrieving Results for Index 444: birjand.
Closest hotel in birjand is مهمان پذیر نوید.
------------
-------End of Search-------


In [83]:
# Showing hotels in area
hotels_df.head()

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,Hotel Escarpas do Lago
88,cidreira,BR,-30.17,-50.22,Hotel Castelo
138,morondava,MG,-20.30,44.28,Kimony Resort Hotel
139,vaini,IN,15.34,74.49,Dandeli Aura Jungle Stay
176,nador,MA,35.17,-2.93,Hotel Maghreb El Arabi


In [84]:
# Creating hotel labels
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Storing the dataframe row
hotels = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]]

In [85]:
# Add marker layer and info box content ontop of heat map
labels = gmaps.marker_layer(locations, info_box_content = hotels)

# Add the layer to the map
fig.add_layer(labels)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))